In [ ]:
import pandas as pd
datos=pd.read_excel("E-Commerce_train.xlsx")
datos.head()

In [2]:
from sklearn.preprocessing import MinMaxScaler

#instanciamos nuestro objeto con un rango correspondiende al del descuento obrecido en tantos por uno, ignorando la copia
scaler = MinMaxScaler(feature_range=(.01,.65), copy=False)

scaler.fit(datos[['Discount_offered']])

datos['Discount_offered'] = scaler.transform(datos[['Discount_offered']])
from sklearn.preprocessing import MinMaxScaler

#instanciamos nuestro objeto con un rango de 1 a 10 ignorando la copia
scaler = MinMaxScaler(feature_range=(1,10), copy=False)

scaler.fit(datos[['Weight_in_gms']])

datos['Weight_in_gms'] = scaler.transform(datos[['Weight_in_gms']])

seleccion=["Discount_offered","Weight_in_gms"]

objetivo=datos["Reached.on.Time_Y.N"]

variables=datos[seleccion]

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(min_samples_split=0.08, n_estimators= 15000)

clf = clf.fit(variables, objetivo)

In [6]:
apredecir=pd.read_excel("E-Commerce_test.xlsx")

In [8]:
from sklearn.preprocessing import MinMaxScaler

#instanciamos nuestro objeto con un rango correspondiende al del descuento obrecido en tantos por uno, ignorando la copia
scaler = MinMaxScaler(feature_range=(.01,.65), copy=False)

scaler.fit(apredecir[['Discount_offered']])

apredecir['Discount_offered'] = scaler.transform(apredecir[['Discount_offered']])
from sklearn.preprocessing import MinMaxScaler

#instanciamos nuestro objeto con un rango de 1 a 10 ignorando la copia
scaler = MinMaxScaler(feature_range=(1,10), copy=False)

scaler.fit(apredecir[['Weight_in_gms']])

apredecir['Weight_in_gms'] = scaler.transform(apredecir[['Weight_in_gms']])

seleccion=["Discount_offered","Weight_in_gms"]

predecirVariables=apredecir[seleccion]

In [10]:
salida=clf.predict(predecirVariables)

In [18]:
salida=pd.DataFrame(salida,columns=["pred"])

In [19]:
salida.to_csv("sypten.csv",index=False)